In [7]:
import numpy
import math
from numpy.linalg import inv 

In [8]:
def FLDA(X,Y):
    f = X.shape[1]  #data dimension
    C0 = numpy.array([X[i] for i in range(X.shape[0]) if Y[i] == 0])   #separating class 0
    C1 = numpy.array([X[i] for i in range(X.shape[0]) if Y[i] == 1])   #separating class 1
    M0 = numpy.sum(C0,axis = 0)/C0.shape[0]                            #mean of Class 0
    M1 = numpy.sum(C1,axis = 0)/C1.shape[0]                            #mean of class 1
    M0 = numpy.reshape(M0,(f,1))                                        
    M1 = numpy.reshape(M1,(f,1))
    C0 = numpy.reshape(C0,(C0.shape[0],C0.shape[1],1))                 
    C1 = numpy.reshape(C1,(C1.shape[0],C1.shape[1],1))
    SW0 = numpy.sum(numpy.array([(C0[i]-M0)@(C0[i]-M0).T for i in range(C0.shape[0])]), axis = 0)
    SW1 = numpy.sum(numpy.array([(C1[i]-M1)@(C1[i]-M1).T for i in range(C1.shape[0])]), axis = 0)
    SW = SW0 + SW1
    W = inv(SW)@(M1-M0)
    
    b = -(W.T@(M1+M0) + (W.T@W)*W.T@(SW0@W - SW1@W))/2
    return W.reshape(X.shape[1]),b[0][0]

In [2]:
import csv
import numpy
# reading csv file to extract data into numpy array
with open('Medical_data.csv', 'r') as f:
    medical = list(csv.reader(f, delimiter=','))
    med_data = numpy.array(medical)

# separating feature and label
X = med_data[1:1000,1:].astype('float')   # feature matrix
Y = med_data[1:1000,0]  # label of input
classes = ['HEALTHY','SURGERY','MEDICATION']

In [5]:
def multiclassFLDA(X,Y,classes):
    f = X.shape[1]
    n = X.shape[0]
    c = len(classes)
    W = numpy.zeros((c,f))
    b = numpy.zeros((c))
    for i in range(c):
        Ty = numpy.zeros((n))
        for j in range(n):
            if Y[j] == classes[i]:
                Ty[j] = 1
            else:
                Ty[j] = 0
        W[i],b[i] = FLDA(X,Ty)
    return W,b

In [9]:
W,b = multiclassFLDA(X,Y,classes)

In [14]:
def accuracy(X,Y,W,b,classes):
    c = len(classes)         #number of classes
    n = X.shape[0]           #number of data samples
    f = W.shape[1]           #input dimension
    acc = 0                  #number of correctly classified inputs
    
    # implementing soft-max to calculate predicted class
    g = numpy.zeros(c)     # probability for each class
    for i in range(n):
        g = numpy.exp((W@X[i].reshape(f,1)).reshape(c)+b)
        y_predicted = numpy.where(g == max(g))
        if (y_predicted[0] == classes.index(Y[i])):
            acc += 1
    
    return acc/X.shape[0]

In [15]:
acc = accuracy(X,Y,W,b,classes)
print (acc*100)

80.78078078078079


In [ ]:
# code may be improved